In [18]:
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

train_set = './data/KDDTrain+.txt'
#test_set = 'KDDTest+.txt'

train_df = pd.read_csv(train_set, header=None)
#test_df = pd.read_csv(test_set, header=None)

train_data = train_df.iloc[:, :-1]
#test_data = test_df.iloc[:, :-1]

#replace not 'normal' labels with 'attack'
train_data.iloc[:, -1] = np.where(train_data.iloc[:, -1] != 'normal', 'attack', train_data.iloc[:, -1])
#test_data.iloc[:, -1] = np.where(test_data.iloc[:, -1] != 'normal', 'attack', test_data.iloc[:, -1])

#explicitly assigning numerical labels to 'attack' and 'normal'
train_data.iloc[:, -1] = np.where(train_data.iloc[:, -1] != 'normal', 1, 0)
#test_data.iloc[:, -1] = np.where(test_data.iloc[:, -1] != 'normal', 1, 0)

for col in train_data.columns[train_data.dtypes == object]:
    encoder = LabelEncoder()
    train_data[col] = encoder.fit_transform(train_data[col])
    #test_data[col] = encoder.transform(test_data[col])

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Sampler
from torch.utils.data import Sampler

class BalancedBatchSampler(Sampler):
    def __init__(self, dataset, batch_size):
        self.dataset = dataset
        self.batch_size = batch_size
        self.num_samples = len(dataset)
        self.indices = list(range(self.num_samples))

        #indices for each class
        self.positive_indices = [i for i in range(self.num_samples) if self.dataset[i][1] == 1]
        self.negative_indices = [i for i in range(self.num_samples) if self.dataset[i][1] == 0]

        #number of samples for each class in each batch
        self.num_positive_samples = (batch_size + 1) // 2
        self.num_negative_samples = batch_size - self.num_positive_samples

    def __iter__(self):
        #shuffle
        np.random.shuffle(self.positive_indices)
        np.random.shuffle(self.negative_indices)
        for i in range(0, self.num_samples, self.batch_size):
            positive_batch_indices = self.positive_indices[i:i+self.num_positive_samples]
            negative_batch_indices = self.negative_indices[i:i+self.num_negative_samples]

            batch_indices = positive_batch_indices + negative_batch_indices
            if len(batch_indices) < self.batch_size:
                num_missing_samples = self.batch_size - len(batch_indices)
                additional_negative_indices = self.negative_indices[:num_missing_samples]
                batch_indices += additional_negative_indices

            yield batch_indices

    def __len__(self):
        return (self.num_samples + self.batch_size - 1) // self.batch_size


#splitting datasets
X_train, y_train = train_data.iloc[:, :-1], train_data.iloc[:, -1]
#X_test, y_test = test_data.iloc[:, :-1], test_data.iloc[:, -1]

#normalization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

from sklearn.model_selection import train_test_split
#X_train_scaled, X_val_scaled, y_train, y_val = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)
test_size=0.2
val_split=0.2
X_train_temp, X_test, y_train_temp, y_test = train_test_split(X_train_scaled, y_train, test_size=test_size, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_temp, y_train_temp, test_size=val_split/(1-test_size), random_state=42)

train_dataset = TensorDataset(torch.tensor(X_train).float(), torch.tensor(y_train.values).long())
val_dataset = TensorDataset(torch.tensor(X_val).float(), torch.tensor(y_val.values).long())
test_dataset = TensorDataset(torch.tensor(X_test).float(), torch.tensor(y_test.values).long())

batch_size = 64

train_balanced_sampler = BalancedBatchSampler(train_dataset, batch_size)
val_balanced_sampler = BalancedBatchSampler(val_dataset, batch_size)
test_balanced_sampler = BalancedBatchSampler(test_dataset, batch_size)

train_loader = DataLoader(train_dataset, batch_sampler=train_balanced_sampler)
val_loader = DataLoader(val_dataset, batch_sampler=val_balanced_sampler)
test_loader = DataLoader(test_dataset, batch_sampler=test_balanced_sampler)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [20]:
print("Size of train dataset:", len(train_loader.dataset))
print("Size of test dataset:", len(test_loader.dataset))
print("Batch size:", batch_size)

num_positives_train = sum(1 for _, label in train_dataset if label == 1)
num_positives_test = sum(1 for _, label in test_dataset if label == 1)

print("Number of positive samples in train dataset:", num_positives_train)
print("Number of positive samples in test dataset:", num_positives_test)

Size of train dataset: 75583
Size of test dataset: 25195
Batch size: 64
Number of positive samples in train dataset: 35128
Number of positive samples in test dataset: 11773


In [21]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

file_path = './data/KDDTrain+.txt'
column_names = [
    "duration", "protocol_type", "service", "flag", "src_bytes", "dst_bytes",
    "land", "wrong_fragment", "urgent", "hot", "num_failed_logins", "logged_in",
    "num_compromised", "root_shell", "su_attempted", "num_root", "num_file_creations",
    "num_shells", "num_access_files", "num_outbound_cmds", "is_host_login",
    "is_guest_login", "count", "srv_count", "serror_rate", "srv_serror_rate",
    "rerror_rate", "srv_rerror_rate", "same_srv_rate", "diff_srv_rate",
    "srv_diff_host_rate", "dst_host_count", "dst_host_srv_count",
    "dst_host_same_srv_rate", "dst_host_diff_srv_rate", "dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate", "dst_host_serror_rate", "dst_host_srv_serror_rate",
    "dst_host_rerror_rate", "dst_host_srv_rerror_rate", "label"
]
train_df = pd.read_csv(file_path, header=None, names=column_names)

#train_numeric = train_df.drop(columns=["protocol_type", "service", "flag", "label"])
#sns.pairplot(train_numeric.sample(frac=0.01), diag_kind='kde')
#plt.show()

In [22]:
import torch
import time
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class FFNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FFNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

        #hidden layers
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, hidden_size)
        self.fc5 = nn.Linear(hidden_size, hidden_size)
        self.fc6 = nn.Linear(hidden_size, hidden_size)
        self.fc7 = nn.Linear(hidden_size, hidden_size)
        self.fc8 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.fc2(out)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.fc3(out)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.fc4(out)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.fc5(out)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.fc6(out)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.fc7(out)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.fc8(out)
        return out

def train_ffnn(model, criterion, optimizer, train_loader, val_loader, num_epochs):
    total_step = len(train_loader)
    for epoch in range(num_epochs):
        model.train()
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if (i+1) % 100 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                       .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
        #Validation
        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            val_accuracy = 100 * correct / total
            print('Validation Accuracy after Epoch {}: {:.2f} %'.format(epoch+1, val_accuracy))

from sklearn.metrics import precision_score, confusion_matrix
def test_ffnn(model, test_loader):
    start_time = time.time()
    with torch.no_grad():
        correct = 0
        total = 0
        predictions = []
        targets = []
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            predictions.extend(predicted.tolist())
            targets.extend(labels.tolist())
        end_time = time.time()
        accuracy = 100 * correct / total
        precision = precision_score(targets, predictions)
        tn, fp, fn, tp = confusion_matrix(targets, predictions).ravel()
        print('Time taken for testing: {:.2f} seconds'.format(end_time - start_time))
        print('Accuracy of the FFNN on the test dataset: {:.2f} %'.format(accuracy))
        print('Precision of the FFNN on the test dataset: {:.2f}'.format(precision))
        print('True Positives (TP):', tp)
        print('True Negatives (TN):', tn)
        print('False Positives (FP):', fp)
        print('False Negatives (FN):', fn)

input_size = X_train_scaled.shape[1]
hidden_size = 128
num_classes = 2
model = FFNN(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5
train_ffnn(model, criterion, optimizer, train_loader,val_loader, num_epochs)
test_ffnn(model, test_loader)

Epoch [1/5], Step [100/1181], Loss: 0.0828
Epoch [1/5], Step [200/1181], Loss: 0.0994
Epoch [1/5], Step [300/1181], Loss: 0.0919
Epoch [1/5], Step [400/1181], Loss: 0.0908
Epoch [1/5], Step [500/1181], Loss: 0.0998
Epoch [1/5], Step [600/1181], Loss: 0.0003
Epoch [1/5], Step [700/1181], Loss: 0.0001
Epoch [1/5], Step [800/1181], Loss: 0.0000
Epoch [1/5], Step [900/1181], Loss: 0.0000
Epoch [1/5], Step [1000/1181], Loss: 0.0001
Epoch [1/5], Step [1100/1181], Loss: 0.0000
Validation Accuracy after Epoch 1: 94.93 %
Epoch [2/5], Step [100/1181], Loss: 0.0246
Epoch [2/5], Step [200/1181], Loss: 0.0603
Epoch [2/5], Step [300/1181], Loss: 0.0769
Epoch [2/5], Step [400/1181], Loss: 0.0222
Epoch [2/5], Step [500/1181], Loss: 0.0394
Epoch [2/5], Step [600/1181], Loss: 0.0004
Epoch [2/5], Step [700/1181], Loss: 0.0000
Epoch [2/5], Step [800/1181], Loss: 0.0001
Epoch [2/5], Step [900/1181], Loss: 0.0000
Epoch [2/5], Step [1000/1181], Loss: 0.0000
Epoch [2/5], Step [1100/1181], Loss: 0.0000
Validat

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

class CNN(nn.Module):
    def __init__(self, input_shape, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv_out_size = self._get_conv_out_size(input_shape)

        self.fc1 = nn.Linear(self.conv_out_size, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        out = self.conv1(x)
        out = self.relu(out)
        out = self.maxpool(out)
        out = self.conv2(out)
        out = self.relu(out)
        out = self.maxpool(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

    def _get_conv_out_size(self, input_shape):
        batch_size = 1
        input_tensor = torch.autograd.Variable(torch.rand(batch_size, *input_shape))
        output_feat = self._forward_features(input_tensor)
        n_size = output_feat.data.view(batch_size, -1).size(1)
        return n_size

    def _forward_features(self, x):
        out = self.conv1(x)
        out = self.relu(out)
        out = self.maxpool(out)
        out = self.conv2(out)
        out = self.relu(out)
        out = self.maxpool(out)
        return out

def train_cnn(model, criterion, optimizer, train_loader, num_epochs):
    total_step = len(train_loader)
    for epoch in range(num_epochs):
        for i, (inputs, labels) in enumerate(train_loader):
            inputs = inputs.unsqueeze(1)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if (i+1) % 100 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                       .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

from sklearn.metrics import precision_score, confusion_matrix

def test_cnn(model, test_loader):
    start_time = time.time()
    with torch.no_grad():
        correct = 0
        total = 0
        predictions = []
        targets = []

        for inputs, labels in test_loader:
            inputs = inputs.unsqueeze(1)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            predictions.extend(predicted.tolist())
            targets.extend(labels.tolist())
        end_time = time.time()
        accuracy = 100 * correct / total
        precision = precision_score(targets, predictions)
        tn, fp, fn, tp = confusion_matrix(targets, predictions).ravel()

        print('Time taken for testing: {:.2f} seconds'.format(end_time - start_time))
        print('Accuracy of the CNN on the test dataset: {:.2f} %'.format(accuracy))
        print('Precision of the CNN on the test dataset: {:.2f}'.format(precision))
        print('True Positives (TP):', tp)
        print('True Negatives (TN):', tn)
        print('False Positives (FP):', fp)
        print('False Negatives (FN):', fn)

batch_size = 64

X_train_tensor = torch.tensor(X_train).float()
y_train_tensor = torch.tensor(y_train.values).long()
X_val_tensor = torch.tensor(X_val).float()
y_val_tensor = torch.tensor(y_val.values).long()
X_test_tensor = torch.tensor(X_test).float()
y_test_tensor = torch.tensor(y_test.values).long()

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

input_shape = (X_train.shape[1],)
num_classes = 2
model = CNN(input_shape, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5
train_cnn(model, criterion, optimizer, train_loader, num_epochs)

test_cnn(model, test_loader)

Epoch [1/5], Step [100/1181], Loss: 0.1313
Epoch [1/5], Step [200/1181], Loss: 0.2287
Epoch [1/5], Step [300/1181], Loss: 0.2420
Epoch [1/5], Step [400/1181], Loss: 0.0586
Epoch [1/5], Step [500/1181], Loss: 0.0768
Epoch [1/5], Step [600/1181], Loss: 0.0163
Epoch [1/5], Step [700/1181], Loss: 0.0601
Epoch [1/5], Step [800/1181], Loss: 0.0313
Epoch [1/5], Step [900/1181], Loss: 0.0531
Epoch [1/5], Step [1000/1181], Loss: 0.0599
Epoch [1/5], Step [1100/1181], Loss: 0.0175
Epoch [2/5], Step [100/1181], Loss: 0.0188
Epoch [2/5], Step [200/1181], Loss: 0.0372
Epoch [2/5], Step [300/1181], Loss: 0.0233
Epoch [2/5], Step [400/1181], Loss: 0.0128
Epoch [2/5], Step [500/1181], Loss: 0.0118
Epoch [2/5], Step [600/1181], Loss: 0.0153
Epoch [2/5], Step [700/1181], Loss: 0.0522
Epoch [2/5], Step [800/1181], Loss: 0.0263
Epoch [2/5], Step [900/1181], Loss: 0.0534
Epoch [2/5], Step [1000/1181], Loss: 0.0276
Epoch [2/5], Step [1100/1181], Loss: 0.0218
Epoch [3/5], Step [100/1181], Loss: 0.0620
Epoch [